In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select

In [2]:

from selenium import webdriver
options = webdriver.ChromeOptions()

# 우리가 데이터를 가져올 웹 페이지입니다.
my_opgg_url = 'https://www.op.gg/summoner/userName=midplayer%20or%20AFK'
options.add_argument('headless')
driver = webdriver.Chrome("/Users/Mgyu/chromedriver", options= options)

In [3]:
# 데이터를 가져올 페이지로 이동합니다.
driver.get(my_opgg_url)

In [4]:
queue_select = Select(driver.find_element_by_css_selector('select.SelectMatchTypes'))

In [5]:
queue_select.select_by_value('aram')
time.sleep(2)

In [6]:
# '더 보기' 버튼이 활성화되어있으면(리스트의 끝까지 도달하지 않았으면) 계속 클릭하여 이전 게임을 모두 로딩합니다.
while True:
    try:
        driver.find_element_by_css_selector('.GameMoreButton').click()
        # 게임 로딩, html 코드 변경까지 여유 시간을 1초 가집니다.
        time.sleep(1)
        
    # 에러가 나면(페이지에서 '더 보기' 버튼이 없을 경우) while문을 탈출합니다.
    except Exception as e:
        pass
        break

In [7]:
# 결과가 들어갈 빈 리스트를 만듭니다.
my_gametime= []
my_champions = []
my_level = []

In [8]:
# 각 게임에 대해 웹 페이지에 기재된 스탯을 찾아서(selector 사용) 결과 리스트에 append하기
for gamelist in driver.find_elements_by_css_selector('div.GameItemList'):
    for game in gamelist.find_elements_by_css_selector('div.GameItemWrap'):
        my_gametime.append(game.find_element_by_css_selector('div.GameStats').
                           find_element_by_css_selector('div.GameLength').text.strip())
        my_champions.append(game.find_element_by_css_selector('div.ChampionName').text.strip())
        my_level.append(game.find_element_by_css_selector('div.Stats').
                           find_element_by_css_selector('div.Level').text.strip())
   

In [9]:
# selenium이 제어하는 크롬을 종료합니다.
driver.quit()

In [10]:
len(my_gametime)

301

In [11]:
# DataFrame으로 변환 후 출력
my_total_df = pd.DataFrame([my_gametime,
                               my_champions,
                                my_level],
                               index = ['gametime','champion','level']).T
my_total_df

,gametime,champion,level
0,17분 38초,요네,레벨17
1,20분 34초,일라오이,레벨18
2,24분 32초,룰루,레벨18
3,12분 20초,트위스티드 페이트,레벨14
4,19분 9초,아펠리오스,레벨17
...,...,...,...
296,24분 44초,제라스,레벨18
297,18분 40초,사일러스,레벨16
298,20분 16초,세나,레벨18
299,24분 31초,모르가나,레벨18
